# ESEF-Berichterstattung in Deutschland: Herausforderungen und Chancen

## Der Python-Code zum Replizieren der Tabellen 1–3

Das Jupyter-Notebook befindet sich unter: https://github.com/esefpaper/onlineappendix/blob/main/code/Tabellen.ipynb

In [52]:
!pip install --force-reinstall numpy==2.2.4 pandas==2.2.3
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 27.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 26.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 22.4 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.1
    Uninstalling pytz-2025.1:
      Successfully uninstalled pytz-2025.1
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2025.2
    Uninstalling tzdata-2025.2:
      Successfully uninstalled tzdata-2025.2
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully

In [50]:
import pandas as pd
import re

In [ ]:
import os
os.makedirs("../results", exist_ok=True)


### Tabelle 1

Tabelle 1 zeigt die ESEF-Konformitätsraten deut-scher kapitalmarktorientierter Unternehmen über den Beobachtungszeitraum von 2020 bis 2023. 

In [11]:
tb1_uregdw_s5 = pd.read_pickle("../data/tb1_uregdw_s5_20250616.p.gz")

In [12]:
def create_tb1_panel(df, byvar, ascending =True):
    # Group by the specified variable
    tb1_de_rows = df.groupby([byvar]).gvkey.nunique().reset_index().rename(columns={"gvkey": "gvkey_nunique"})
    
    # Create the different groupings
    tb1c_01 = df.groupby([byvar]).gvkey.count().reset_index()
    tb1c_02 = df.loc[df.sum_filing_score>=8].groupby([byvar]).gvkey.count().reset_index()
    tb1c_03 = df.loc[df.sum_filing_score>=16].groupby([byvar]).gvkey.count().reset_index()
    tb1c_04 = df.loc[df.sum_filing_score>=32768].groupby([byvar]).gvkey.count().reset_index()
    
    # Merge all the data
    result = pd.merge(tb1_de_rows, tb1c_01, on=byvar, how="left", suffixes=("", "")).fillna(0)
    result = pd.merge(result, tb1c_02, on=byvar, how="left", suffixes=("", "_esef")).fillna(0)
    result["anteil_esef"] = result["gvkey_esef"]/result["gvkey"]
    result["anteil_esef"] = result["anteil_esef"].apply(lambda x: round(x*100, 1))
    
    result = pd.merge(result, tb1c_03, on=byvar, how="left", suffixes=("", "_xbrl")).fillna(0)
    result["anteil_xbrl"] = result["gvkey_xbrl"]/result["gvkey"]
    result["anteil_xbrl"] = result["anteil_xbrl"].apply(lambda x: round(x*100, 1))
    
    result = pd.merge(result, tb1c_04, on=byvar, how="left", suffixes=("", "_complete")).fillna(0)
    result["anteil_complete"] = result["gvkey_complete"]/result["gvkey"]
    result["anteil_complete"] = result["anteil_complete"].apply(lambda x: round(x*100, 1))
    
    result.fillna(0, inplace=True)
    result.sort_values(by=byvar, ascending=ascending, inplace=True)
    result.to_excel(f"../results/tb1_by{byvar}.xlsx")
    return result


In [13]:
# Panel A
tb1_by_formyear = create_tb1_panel(tb1_uregdw_s5, "formyear")
tb1_by_formyear

,formyear,gvkey_nunique,gvkey,gvkey_esef,anteil_esef,gvkey_xbrl,anteil_xbrl,gvkey_complete,anteil_complete
0,2020,596,596,354,59.4,330,55.4,317,53.2
1,2021,595,595,410,68.9,405,68.1,362,60.8
2,2022,572,572,397,69.4,392,68.5,355,62.1
3,2023,492,492,368,74.8,362,73.6,330,67.1


In [14]:
# table 1 2020--2023 ESEF Sample period
tb1_uregdw_s5["2020--2023"] = tb1_uregdw_s5.formyear.apply(lambda x: True if x>=2020 & x<=2024 else False)

byvar = "2020--2023"
tb1_esef = create_tb1_panel(tb1_uregdw_s5, byvar, ascending=False)
tb1_esef

,2020--2023,gvkey_nunique,gvkey,gvkey_esef,anteil_esef,gvkey_xbrl,anteil_xbrl,gvkey_complete,anteil_complete
0,True,610,2255,1529,67.8,1489,66.0,1364,60.5


In [15]:
byvar = "FSE_Label"
tb1_by_FSE_label = create_tb1_panel(tb1_uregdw_s5, byvar)
tb1_by_FSE_label

,FSE_Label,gvkey_nunique,gvkey,gvkey_esef,anteil_esef,gvkey_xbrl,anteil_xbrl,gvkey_complete,anteil_complete
0,1. DAX40,42,163,157,96.3,157,96.3,155,95.1
1,2. Prime Standard,208,806,707,87.7,701,87.0,681,84.5
2,3. SME,82,299,220,73.6,213,71.2,186,62.2
3,4. Aktuell nicht als Aktie an FWB,282,987,445,45.1,418,42.4,342,34.7


In [16]:
tb1_uregdw_s5["DE_ISIN"] = tb1_uregdw_s5["isin"].str.startswith("DE")
tb1_uregdw_s5["DE_ISIN"] = tb1_uregdw_s5["DE_ISIN"].fillna(True)

byvar = "DE_ISIN"
tb1_de_isin = create_tb1_panel(tb1_uregdw_s5, byvar, ascending=False)
tb1_de_isin

/var/folders/mz/hx4rhsms565c3zkfh7qx8xs00000gn/T/ipykernel_43243/187706699.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tb1_uregdw_s5["DE_ISIN"] = tb1_uregdw_s5["DE_ISIN"].fillna(True)


,DE_ISIN,gvkey_nunique,gvkey,gvkey_esef,anteil_esef,gvkey_xbrl,anteil_xbrl,gvkey_complete,anteil_complete
1,True,590,2189,1492,68.2,1452,66.3,1330,60.8
0,False,20,66,37,56.1,37,56.1,34,51.5


In [17]:
byvar = "mcap_g4"
tb1_de_mcap_beg = create_tb1_panel(tb1_uregdw_s5, byvar, ascending=False)
tb1_de_mcap_beg

,mcap_g4,gvkey_nunique,gvkey,gvkey_esef,anteil_esef,gvkey_xbrl,anteil_xbrl,gvkey_complete,anteil_complete
3,MCap4,160,528,360,68.2,353,66.9,312,59.1
2,MCap3,159,476,359,75.4,357,75.0,342,71.8
1,MCap2,219,661,433,65.5,420,63.5,380,57.5
0,MCap1,186,590,377,63.9,359,60.8,330,55.9


In [18]:
byvar = "gsec_type"
tb1_de_mcap_beg = create_tb1_panel(tb1_uregdw_s5, byvar, ascending=True)
tb1_de_mcap_beg

,gsec_type,gvkey_nunique,gvkey,gvkey_esef,anteil_esef,gvkey_xbrl,anteil_xbrl,gvkey_complete,anteil_complete
0,10 Energie,7,26,13,50.0,13,50.0,12,46.2
1,15 Roh- und Grundstoffe,32,115,94,81.7,89,77.4,77,67.0
2,20 Industrie,136,519,398,76.7,395,76.1,373,71.9
3,25 Verbraucher Diskretionäre,90,329,223,67.8,218,66.3,196,59.6
4,30 Verbraucher Staples,25,92,44,47.8,44,47.8,44,47.8
5,35 Gesundheitswesen,61,222,145,65.3,135,60.8,124,55.9
6,40 Finanzen,53,206,160,77.7,152,73.8,126,61.2
7,45 Informationstechnologie,109,386,201,52.1,200,51.8,194,50.3
8,50 Telekommunikation,46,167,111,66.5,108,64.7,101,60.5
9,55 Energieversorgung,16,61,42,68.9,41,67.2,36,59.0


### Tabelle 2

Tabelle 2 analysiert die Verwendung von Standard-IFRS-Taxonomie-Tags und firmenspezifischen Erweiterungstags in den ESEF-Berichten. Es ist zu beachten, dass die Methodik das Zählen der Anzahl der Konzepte unter der XBRL-Taxonomie umfasst, nicht jedoch die Anzahl der Fakten.

In [19]:
data_tb2 = pd.read_pickle("../data/data_tb2_20250616.p.gz", compression="gzip")
data_tb2i = pd.read_pickle("../data/data_tb2i_20250616.p.gz", compression="gzip")

In [20]:
def create_tb2_panel(data_tb2, byvar, ascending=True):
    # Group by the specified variable
    tb2a_rows = data_tb2.groupby([byvar]).gvkey.nunique().reset_index().rename(columns={"gvkey": "gvkey_nunique"})

    tb2a_01 = data_tb2.groupby([byvar, "gvkey"]).concept_name.nunique().reset_index().groupby(byvar).concept_name.mean().apply(lambda x: round(x, 1)).reset_index()
    tb2a_02 = data_tb2.loc[ ~data_tb2.concept_is_extended].groupby([byvar, "gvkey"]).concept_name.nunique().reset_index().groupby(byvar).concept_name.mean().apply(lambda x: round(x, 1)).reset_index()
    tb2a_03 = data_tb2.loc[ data_tb2.concept_is_extended].groupby([byvar, "gvkey"]).concept_name.nunique().reset_index().groupby(byvar).concept_name.mean().apply(lambda x: round(x, 1)).reset_index()

    tb2a = pd.merge(tb2a_01, tb2a_02, on=byvar, how="left", suffixes=("", "_ifrs")).fillna(0)
    tb2a = pd.merge(tb2a, tb2a_03, on=byvar, how="left", suffixes=("", "_erw")).fillna(0)

    tb2a.fillna(0, inplace=True)
    tb2a["concept_name"] = tb2a["concept_name_ifrs"] + tb2a["concept_name_erw"]
    tb2a["anteil_ifrs"] = tb2a["concept_name_ifrs"]/tb2a["concept_name"]
    tb2a["anteil_ifrs"] = tb2a["anteil_ifrs"].apply(lambda x: round(x*100, 1))

    tb2a["anteil_erw"] = tb2a["concept_name_erw"]/tb2a["concept_name"]
    tb2a["anteil_erw"] = tb2a["anteil_erw"].apply(lambda x: round(x*100, 1))


    tb2a.sort_values(by=byvar, ascending=ascending, inplace=True)
    tb2a.to_excel(f"../results/tb2_{byvar}.xlsx")
    tb2a

    return tb2a


In [22]:
#formyear
tb2_by_formyear = create_tb2_panel(data_tb2, "formyear")
tb2_by_formyear

,formyear,concept_name,concept_name_ifrs,concept_name_erw,anteil_ifrs,anteil_erw
0,2020,142.8,127.8,15.0,89.5,10.5
1,2021,143.8,128.5,15.3,89.4,10.6
2,2022,266.8,251.7,15.1,94.3,5.7
3,2023,282.5,267.8,14.7,94.8,5.2


In [21]:
#statement_type / TagType
tb2_by_statement_type = create_tb2_panel(data_tb2i, "statement_type")
tb2_by_statement_type

,statement_type,concept_name,concept_name_ifrs,concept_name_erw,anteil_ifrs,anteil_erw
0,0. General,2.3,2.3,0.0,100.0,0.0
1,1. Bilanz,42.7,35.7,7.0,83.6,16.4
2,2. GuV,25.5,19.9,5.6,78.0,22.0
3,3. GKV/UKV,17.7,12.9,4.8,72.9,27.1
4,4. Kapitalflussrechnung,34.5,24.0,10.5,69.6,30.4
5,5. EK-Veränderungsrechnung,20.6,16.1,4.5,78.2,21.8
6,6. Nettovermögensänderung,1.0,1.0,0.0,100.0,0.0
7,7. Anhang,164.7,160.1,4.6,97.2,2.8


In [23]:
#FSE_Label
tb2_by_FSE_Label = create_tb2_panel(data_tb2, "FSE_Label")
tb2_by_FSE_Label

,FSE_Label,concept_name,concept_name_ifrs,concept_name_erw,anteil_ifrs,anteil_erw
0,1. DAX40,364.6,316.8,47.8,86.9,13.1
1,2. Prime Standard,308.4,284.9,23.5,92.4,7.6
2,3. SME,263.0,243.5,19.5,92.6,7.4
3,4. Aktuell nicht als Aktie an FWB,266.9,242.0,24.9,90.7,9.3


In [24]:
#DE_ISIN
data_tb2["DE_ISIN"] = data_tb2["isin"].str.startswith("DE")
tb2_by_gsec_type = create_tb2_panel(data_tb2, "DE_ISIN", ascending=False)
tb2_by_gsec_type

,DE_ISIN,concept_name,concept_name_ifrs,concept_name_erw,anteil_ifrs,anteil_erw
1,True,309.3,283.3,26.0,91.6,8.4
0,False,287.4,258.8,28.6,90.0,10.0


In [25]:
#mcap_g4
tb2_by_mcap_g4 = create_tb2_panel(data_tb2, "mcap_g4", ascending=False)
tb2_by_mcap_g4

,mcap_g4,concept_name,concept_name_ifrs,concept_name_erw,anteil_ifrs,anteil_erw
3,MCap4,302.4,266.7,35.7,88.2,11.8
2,MCap3,284.7,262.7,22.0,92.3,7.7
1,MCap2,261.0,242.3,18.7,92.8,7.2
0,MCap1,261.0,243.8,17.2,93.4,6.6


In [26]:
#gsec_type
tb2_by_gsec_type = create_tb2_panel(data_tb2, "gsec_type")
tb2_by_gsec_type

,gsec_type,concept_name,concept_name_ifrs,concept_name_erw,anteil_ifrs,anteil_erw
0,10 Energie,320.3,282.5,37.8,88.2,11.8
1,15 Roh- und Grundstoffe,325.3,296.3,29.0,91.1,8.9
2,20 Industrie,305.3,280.8,24.5,92.0,8.0
3,25 Verbraucher Diskretionäre,281.3,258.9,22.4,92.0,8.0
4,30 Verbraucher Staples,305.6,279.7,25.9,91.5,8.5
5,35 Gesundheitswesen,291.2,267.6,23.6,91.9,8.1
6,40 Finanzen,322.6,274.2,48.4,85.0,15.0
7,45 Informationstechnologie,279.3,263.6,15.7,94.4,5.6
8,50 Telekommunikation,270.3,254.1,16.2,94.0,6.0
9,55 Energieversorgung,323.1,280.7,42.4,86.9,13.1


### Tabelle 3

Tabelle 3 analysiert die Verwendung von XBRL-Markierungen im Anhang zum Jahresabschluss. Sie zeigt die Anzahl der Unternehmen und Berichte, die bestimmte Anhangangaben taggen, sowie die durchschnittliche Anzahl der Tags pro Anhangang-abe und den Anteil der Text- und Zahlentags. 

In [43]:
data_tb2i["reportkey"] = data_tb2i.gvkey.astype(str)+data_tb2i.formyear.astype(str)
data_tb3 = data_tb2i.loc[data_tb2i.statement_type=="7. Anhang"].copy()

In [44]:
byvar = "TopConcept"
tb2a_01 = data_tb3.groupby(["TopConcept", "gvkey"]).concept_name.nunique().reset_index().groupby("TopConcept").concept_name.mean().apply(lambda x: round(x, 1)).reset_index()
tb2a_01.head(10)

,TopConcept,concept_name
0,"[800100] Notes - Subclassifications of assets,...",10.9
1,[800200] Notes - Analysis of income and expense,3.9
2,"[800300] Notes - Statement of cash flows, addi...",8.0
3,[800400] Notes - Statement of changes in equit...,2.6
4,[800500] Notes - List of notes,85.5
5,[800610] Notes - List of material accounting p...,49.1
6,[810000] Notes - Corporate information and sta...,10.7
7,"[811000] Notes - Accounting policies, changes ...",1.7
8,[813000] Notes - Interim financial reporting,1.0
9,[815000] Notes - Events after reporting period,2.0


In [45]:
data_tb3["type_notetag_text"] = data_tb3.Type_y.astype(str).str.contains("Text|Abstract|Axis|Disclsoure|Lineitems|Textblock")
data_tb3.type_notetag_text = data_tb3.type_notetag_text.fillna(True)
data_tb3.loc[data_tb3.TopType.fillna("").str.contains("800500|800610")].type_notetag_text.value_counts()

tb2a_02 = data_tb3.loc[ ~data_tb3.type_notetag_text].groupby([byvar, "gvkey"]).concept_name.nunique().reset_index().groupby(byvar).concept_name.mean().apply(lambda x: round(x, 1)).reset_index()


In [47]:
tb3_firms = data_tb3.groupby(["TopConcept"]).gvkey.nunique().reset_index().rename(columns={"gvkey": "nfirms"})
tb3_reportkeys = data_tb3.groupby(["TopConcept"]).reportkey.nunique().reset_index().rename(columns={"reportkey": "nreports"})
tb3_rows = pd.merge(tb3_firms, tb3_reportkeys, on="TopConcept", how="left", suffixes=("", "_")).fillna(0)

tb3_rows = pd.merge(tb3_rows, tb2a_01, on="TopConcept", how="left", suffixes=("", "")).fillna(0)
tb3_rows = pd.merge(tb3_rows, tb2a_02, on="TopConcept", how="left", suffixes=("", "_num")).fillna(0)
tb3_rows.sort_values(by="nreports", ascending=False, inplace=True)


In [48]:
tb2a_03 = data_tb3.loc[ data_tb3.type_notetag_text].groupby([byvar, "gvkey"]).concept_name.nunique().reset_index().groupby(byvar).concept_name.mean().apply(lambda x: round(x, 1)).reset_index()

tb3 = pd.merge(tb3_rows, tb2a_03, on="TopConcept", how="left", suffixes=("", "_text")).fillna(0)

tb3.fillna(0, inplace=True)
tb3.head(10)


,TopConcept,nfirms,nreports,concept_name,concept_name_num,concept_name_text
0,[810000] Notes - Corporate information and sta...,410,1357,10.7,2.4,9.4
1,"[800100] Notes - Subclassifications of assets,...",410,1353,10.9,10.8,1.5
2,"[800300] Notes - Statement of cash flows, addi...",404,1320,8.0,8.0,0.0
3,[800200] Notes - Analysis of income and expense,370,1204,3.9,3.9,1.5
4,[800400] Notes - Statement of changes in equit...,382,1182,2.6,2.6,1.0
5,[800500] Notes - List of notes,359,648,85.5,0.0,85.5
6,[800610] Notes - List of material accounting p...,359,648,49.1,0.0,49.1
7,[832610] Notes - Leases,200,621,2.1,2.1,1.6
8,[851100] Notes - Cash flow statement,162,418,1.6,1.5,1.2
9,[822390] Notes - Financial instruments,69,157,1.3,1.2,1.2


In [51]:

tb3["concept_name"] = tb3["concept_name_text"] + tb3["concept_name_num"]

tb3["anteil_num"] = tb3["concept_name_num"]/tb3["concept_name"]
tb3["anteil_num"] = tb3["anteil_num"].apply(lambda x: round(x*100, 1))
tb3["anteil_text"] = tb3["concept_name_text"]/tb3["concept_name"]
tb3["anteil_text"] = tb3["anteil_text"].apply(lambda x: round(x*100, 1))

tb3["TopConcept"] = tb3["TopConcept"].apply(lambda x: re.sub("^\[\d{6}\]\s?","", str(x)))

tb3.sort_values(by="nreports", ascending=False, inplace=True)
tb3[["TopConcept", "nfirms", "nreports", "concept_name", "concept_name_text", "anteil_text", "concept_name_num",  "anteil_num"]].to_excel(f"../results/tb3_TopConcept.xlsx")
tb3[["TopConcept", "nfirms", "nreports", "concept_name", "concept_name_text", "anteil_text", "concept_name_num",  "anteil_num"]]



,TopConcept,nfirms,nreports,concept_name,concept_name_text,anteil_text,concept_name_num,anteil_num
0,Notes - Corporate information and statement of...,410,1357,11.8,9.4,79.7,2.4,20.3
1,"Notes - Subclassifications of assets, liabilit...",410,1353,12.3,1.5,12.2,10.8,87.8
2,"Notes - Statement of cash flows, additional di...",404,1320,8.0,0.0,0.0,8.0,100.0
3,Notes - Analysis of income and expense,370,1204,5.4,1.5,27.8,3.9,72.2
4,"Notes - Statement of changes in equity, additi...",382,1182,3.6,1.0,27.8,2.6,72.2
5,Notes - List of notes,359,648,85.5,85.5,100.0,0.0,0.0
6,Notes - List of material accounting policy inf...,359,648,49.1,49.1,100.0,0.0,0.0
7,Notes - Leases,200,621,3.7,1.6,43.2,2.1,56.8
8,Notes - Cash flow statement,162,418,2.7,1.2,44.4,1.5,55.6
9,Notes - Financial instruments,69,157,2.4,1.2,50.0,1.2,50.0
